## Training with Validation Split and CNN with VGG Net Model Architecture

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
#import gdal
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate, Reshape
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
#import tifffile
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Activation, BatchNormalization, Concatenate
from tensorflow.keras.applications import VGG19
from tensorflow.keras import backend as K

2023-05-25 13:26:25.043030: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 13:26:25.921727: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


2023-05-25 12:43:12.779742: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-25 12:43:12.839462: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
!pwd

/home/ssabat/code/SiddharthSabat/solar_roof_segmentation


In [5]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available:", len(physical_devices))

Num GPUs Available: 0


In [6]:
import os
import shutil

def move_files(source_folder, destination_folder, file_prefix):
  # Get a list of all files in the source folder
  files = os.listdir(source_folder)

  # Create the destination folder if it doesn't exist
  if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)
    
  # Iterate over each file
  for file_name in files:
    # Check if the file name starts with the specified prefix
    if file_name.startswith(file_prefix):
      #print(file_name)
      # Construct the full path of the source file
      source_path = os.path.join(source_folder, file_name)

      # Construct the full path of the destination file
      destination_path = os.path.join(destination_folder, file_name)

      # Move the file to the destination folder
      shutil.move(source_path, destination_path)
      #print(f"Moved {file_name} to {destination_folder}")

In [ ]:
# Usage example
source_folder = 'data/test_patch'
destination_folder = 'data/test_patch/bellingham'
file_prefix = 'bellingham'

move_files(source_folder, destination_folder, file_prefix)


In [ ]:
# Usage example
source_folder = 'data/test_patch'
destination_folder = 'data/test_patch/bloomington'
file_prefix = 'bloomington'

move_files(source_folder, destination_folder, file_prefix)


In [9]:
# Usage example
source_folder = 'data/test_patch'
destination_folder = 'data/test_patch/innsbruck'
file_prefix = 'innsbruck'

move_files(source_folder, destination_folder, file_prefix)


In [10]:
# Usage example
source_folder = 'data/test_patch'
destination_folder = 'data/test_patch/sfo'
file_prefix = 'sfo'

move_files(source_folder, destination_folder, file_prefix)


In [11]:
# Usage example
source_folder = 'data/test_patch'
destination_folder = 'data/test_patch/tyrol'
file_prefix = 'tyrol'

move_files(source_folder, destination_folder, file_prefix)


## Load the patched images using Image data generator with validation splits

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the paths to your image data directories
x_train_dir = 'data/train_patch/'
y_train_dir = 'data/train_label_patch/'

# Set the image and batch size
image_size = (512, 512)  # Target image size for resizing
batch_size = 32

# Data augmentation and normalization configuration for x_train
x_train_generator = ImageDataGenerator(
    rescale=1.0 / 255,  # Scale pixel values to [0, 1]
    rotation_range=20,  # Random rotation between -20 and 20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by 20% of the width
    height_shift_range=0.2,  # Randomly shift images vertically by 20% of the height
    horizontal_flip=True  # Randomly flip images horizontally
    ,validation_split=0.2
)

# Create an image generator for x_train
x_train_image_generator = x_train_generator.flow_from_directory(
    x_train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode=None,  # Set to None to return only input images
    shuffle=False  # Set to False to ensure x_train and y_train match
    ,subset='training'
)
x_val = x_train_generator.flow_from_directory(
    x_train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode=None,  # Set to None to return only input images
    #class_mode='binary',
    subset='validation',  # Subset for validation data
    shuffle=False
)

# Data normalization configuration for y_train
y_train_generator = ImageDataGenerator(
    rescale=1.0 / 255  # Scale pixel values to [0, 1]
    ,validation_split=0.2
)

# Create an image generator for y_train
y_train_image_generator = y_train_generator.flow_from_directory(
    y_train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode=None,  # Set to None to return only input images
    shuffle=False  # Set to False to ensure x_train and y_train match
    ,subset='training'
)

y_val = y_train_generator.flow_from_directory(
    y_train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode=None,  # Set to None to return only input images
    subset='validation',  # Subset for validation data
    shuffle=False
)

# Ensure x_train and y_train generators have the same number of samples
assert len(x_train_image_generator) == len(y_train_image_generator), "Mismatch in x_train and y_train data"
train_data_generator = zip(x_train_image_generator, y_train_image_generator)
val_data_generator =   zip(x_val, y_val)

Found 46660 images belonging to 5 classes.
Found 11660 images belonging to 5 classes.
Found 46660 images belonging to 5 classes.
Found 11660 images belonging to 5 classes.


### Model Architecture and Custom Loss

In [5]:
def loss(y_true, y_pred):
    def dice_loss(y_true, y_pred):
        y_pred = tf.math.sigmoid(y_pred)
        numerator = 2 * tf.reduce_sum(y_true * y_pred)
        denominator = tf.reduce_sum(y_true + y_pred)
        return 1 - numerator / denominator
    y_true = tf.cast(y_true, tf.float32)
    cross_entropy_loss = tf.nn.sigmoid_cross_entropy_with_logits(y_true, y_pred)
    total_loss = cross_entropy_loss + dice_loss(y_true, y_pred)
    return tf.reduce_mean(total_loss)

In [6]:
def iou_metric(y_true, y_pred):
    y_pred = tf.math.round(y_pred)
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred) - intersection
    iou = intersection / (union + K.epsilon())
    return iou

In [7]:
class UNet:
    def __init__(self, input_shape):
        self.input_shape = input_shape
        self.model = self.build_vgg19_unet()
    def conv_block(self, input, num_filters):
        x = Conv2D(num_filters, 3, padding="same")(input)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = Conv2D(num_filters, 3, padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        return x
    def decoder_block(self, input, skip_features, num_filters):
        x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
        x = Concatenate()([x, skip_features])
        x = self.conv_block(x, num_filters)
        return x
    def build_vgg19_unet(self):
        """ Input """
        inputs = Input(self.input_shape)
        """ Pre-trained VGG19 Model """
        vgg19 = VGG19(include_top=False, weights="imagenet", input_tensor=inputs)
        """ Encoder """
        s1 = vgg19.get_layer("block1_conv2").output       ## (512 x 512)
        s2 = vgg19.get_layer("block2_conv2").output       ## (256 x 256)
        s3 = vgg19.get_layer("block3_conv4").output       ## (128 x 128)
        s4 = vgg19.get_layer("block4_conv4").output       ## (64 x 64)
        """ Bridge """
        b1 = vgg19.get_layer("block5_conv4").output       ## (32 x 32)
        """ Decoder """
        d1 = self.decoder_block(b1, s4, 512)              ## (64 x 64)
        d2 = self.decoder_block(d1, s3, 256)              ## (128 x 128)
        d3 = self.decoder_block(d2, s2, 128)              ## (256 x 256)
        d4 = self.decoder_block(d3, s1, 64)               ## (512 x 512)
        """ Output """
        outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)
        model = Model(inputs, outputs, name="VGG19_U-Net")
        return model
    
    def compile(self, optimizer='adam'):
        self.model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
        
    def train(self, train_data_generator, val_data_generator,steps_per_epoch, epochs, batch_size):
        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=loss, metrics=['accuracy', iou_metric])
        lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.0001, verbose=1)
        #self.model.fit(x, y, epochs=epochs, batch_size=batch_size, validation_split = 0.2 , callbacks=[lr_reducer])
        #self.model.fit(train_data_generator, steps_per_epoch=len(x_train_image_generator), epochs=2, batch_size = 32, verbose=1, validation_split = 0.2)
               
        # Use the generator to train the model
        self.model.fit(train_data_generator, validation_data = val_data_generator, steps_per_epoch=steps_per_epoch, epochs=1, batch_size = batch_size, verbose=1, callbacks=[lr_reducer])
        
    def predict(self, x):
        return self.model.predict(x)

In [8]:
steps_per_epoch=len(x_train_image_generator)
input_shape = (512, 512, 3)
batch_size = 32
unet = UNet(input_shape)
# Compile the model
unet.compile(optimizer='adam')
# Print the model summary
unet.model.summary()

Model: "VGG19_U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 512, 512, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 512, 512, 64  36928       ['block1_conv1[0][0]']           
                                )                                                       

In [ ]:
# Train the model
epochs = 2
batch_size = 32
unet.train(train_data_generator, val_data_generator, steps_per_epoch, epochs, batch_size)

2023-05-25 11:02:36.178527: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-25 11:03:16.561225: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4294967296 exceeds 10% of free system memory.
2023-05-25 11:03:36.597063: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4294967296 exceeds 10% of free system memory.


   1/1459 [..............................] - ETA: 70:55:41 - loss: 1.5204 - accuracy: 0.4726 - iou_metric: 0.4312

2023-05-25 11:06:08.404097: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4294967296 exceeds 10% of free system memory.
2023-05-25 11:06:33.039885: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4294967296 exceeds 10% of free system memory.


   2/1459 [..............................] - ETA: 77:02:23 - loss: 1.5090 - accuracy: 0.5370 - iou_metric: 0.3756

2023-05-25 11:09:22.553651: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4294967296 exceeds 10% of free system memory.


  14/1459 [..............................] - ETA: 69:10:23 - loss: 1.4618 - accuracy: 0.7008 - iou_metric: 0.1598